In [1]:
import numpy as np
import pandas as pd
import random

from sklearn import datasets
from sklearn import model_selection
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap

1. В коде из методички реализуйте один или несколько из критериев останова (количество листьев, количество используемых признаков, глубина дерева и т.д.) 

In [2]:
#Генерация датасета
classification_data, classification_labels = datasets.make_classification(n_features = 6, 
                                        n_informative = 2, n_classes = 2, n_redundant=0, 
                                                      n_clusters_per_class=1, random_state=5)

In [3]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [4]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        # подсчет количества объектов разных классов
        classes = {}  # сформируем словарь "класс: количество объектов"
        for label in self.labels:
            if label not in classes:
                classes[label] = 0
            classes[label] += 1
        #  найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
        prediction = max(classes, key=classes.get)
        return prediction 

In [5]:
def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [6]:
def quality(left_labels, right_labels, current_gini):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_gini - p * gini(left_labels) - (1 - p) * gini(right_labels)

In [7]:
def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [8]:
def find_best_split(data, labels, n_features, min_leaf):
    
    #  обозначим минимальное количество объектов в узле
    
    current_gini = gini(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    if n_features > data.shape[1]:
        n_features = data.shape[1]
    
    for index in range(n_features):
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее min_leaf объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_gini)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [9]:
def build_tree(data, labels, n_features, min_leaf):

    quality, t, index = find_best_split(data, labels, n_features, min_leaf)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels, n_features, min_leaf)
    false_branch = build_tree(false_data, false_labels, n_features, min_leaf)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [10]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [11]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [12]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                            classification_labels, test_size = 0.3, random_state = 1)

In [13]:
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [14]:
def get_result(n_features, min_leaf, train_data=train_data,test_data=test_data,
              train_labels=train_labels,test_labels=test_labels):
    tree = build_tree(train_data, train_labels, n_features, min_leaf)
    train_res = predict(train_data, tree)
    result = predict(test_data, tree)
    train_acc = accuracy_metric(train_labels, train_res)
    test_acc = accuracy_metric(test_labels, result)
    print(f'Features: {n_features}, Min_leaf: {min_leaf}')
    print(f'Accuracy for train: {train_acc:.3f}, for test: {test_acc:.3f}')

In [15]:
for i in range(3,6):
    for j in range(5,7):
        get_result(i, j)
        print()

Features: 3, Min_leaf: 5
Accuracy for train: 81.429, for test: 43.333

Features: 3, Min_leaf: 6
Accuracy for train: 80.000, for test: 43.333

Features: 4, Min_leaf: 5
Accuracy for train: 87.143, for test: 26.667

Features: 4, Min_leaf: 6
Accuracy for train: 85.714, for test: 26.667

Features: 5, Min_leaf: 5
Accuracy for train: 87.143, for test: 26.667

Features: 5, Min_leaf: 6
Accuracy for train: 85.714, for test: 26.667



####   Чем больше признаков и минимальное количество листьев, тем хуже качество тестовых данных. Модель переобучается

2*. Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [38]:
data, labels = datasets.make_regression(n_features=2, random_state=5)

In [39]:
data[:5]

array([[ 1.15220477, -0.57751323],
       [ 1.14720208,  0.24981732],
       [ 1.58248112,  0.10960984],
       [-0.32986996,  0.18760323],
       [-0.10061435, -0.33633626]])

In [40]:
labels[:5]

array([ 62.06381583,  77.52011913, 102.45299266, -17.34404931,
       -12.7935693 ])

In [41]:
class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [42]:
class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
        objects = []  # сформируем список объектов в листе
        for label in self.labels:
            objects.append(label)
        #  найдем среднее    
        objects = np.array(objects)
        prediction = objects.mean()
        return prediction   

In [43]:
def means(labels):
    objects = []  
    for label in labels:
        objects.append(label)
    objects = np.array(objects)
    pred = objects.mean()
    return np.mean((labels - pred) ** 2)

In [53]:
def quality(left_labels, right_labels, current_mean):

    # доля выбоки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return current_mean - p * means(left_labels) - (1 - p) * means(right_labels)

In [45]:
def split(data, labels, index, t):
    
    left = np.where(data[:, index] <= t)
    right = np.where(data[:, index] > t)
        
    true_data = data[left]
    false_data = data[right]
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [46]:
def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_leaf = 5

    current_mean = means(labels)

    best_quality = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        t_values = [row[index] for row in data]
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_leaf or len(false_data) < min_leaf:
                continue
            
            current_quality = quality(true_labels, false_labels, current_mean)
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_quality > best_quality:
                best_quality, best_t, best_index = current_quality, t, index

    return best_quality, best_t, best_index

In [47]:
def build_tree(data, labels):

    quality, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if quality == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)
    false_branch = build_tree(false_data, false_labels)

    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [48]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [49]:
def predict(data, tree):
    
    predictions = []
    for obj in data:
        prediction = classify_object(obj, tree)
        predictions.append(prediction)
    return predictions

In [50]:
train_data, test_data, train_labels, test_labels = model_selection.train_test_split(data, 
                                                labels, test_size = 0.3, random_state = 1)

In [54]:
my_tree = build_tree(train_data, train_labels)

In [55]:
train_res = predict(train_data, my_tree)

In [56]:
res = predict(test_data, my_tree)

In [57]:
def mse_(labels, res):
    return np.mean((labels - res) ** 2)

In [58]:
mse_(train_labels, train_res)

218.57831019366

In [59]:
mse_(test_labels, res)

496.16389376803176

In [60]:
def r_2(labels, res):
    return (1 - np.sum((res - labels) ** 2) / np.sum((labels - np.mean(labels))**2))

In [61]:
r_2(train_labels, train_res)

0.9473030504970069

In [62]:
r_2(test_labels, res)

0.8558102546515577